# Código de perfiles y sincronización

En el proceso de desarrollo de código y creación de canales de procesamiento de datos, a menudo se pueden hacer concesiones entre varias implementaciones.
Al principio del desarrollo de su algoritmo, puede resultar contraproducente preocuparse por esas cosas. Como bromeó Donald Knuth: "Deberíamos olvidarnos de las pequeñas eficiencias, digamos aproximadamente el 97% de las veces: la optimización prematura es la raíz de todos los males".

Pero una vez que tenga su código funcionando, puede resultar útil profundizar un poco en su eficiencia.
A veces es útil comprobar el tiempo de ejecución de un comando o conjunto de comandos determinado; otras veces es útil examinar un proceso de varias líneas y determinar dónde reside el cuello de botella en alguna serie complicada de operaciones.
IPython proporciona acceso a una amplia gama de funciones para este tipo de temporización y creación de perfiles de código.
Aquí discutiremos los siguientes comandos mágicos de IPython:

- `%time`: cronometra la ejecución de una sola declaración
- `%timeit`: tiempo de ejecución repetida de una sola declaración para mayor precisión
- `%prun`: Ejecutar código con el generador de perfiles
- `%lprun`: ejecuta código con el generador de perfiles línea por línea
- `%memit`: mide el uso de memoria de una sola declaración
- `%mprun`: ejecuta código con el perfilador de memoria línea por línea

Los últimos cuatro comandos no están incluidos en IPython; para usarlos necesitarás obtener las extensiones `line_profiler` y `memory_profiler`, que discutiremos en las siguientes secciones.

## Fragmentos de código de sincronización: %timeit y %time

Vimos la magia de línea `%timeit` y la magia de celda `%%timeit` en la introducción a las funciones mágicas en [IPython Magic Commands](01.03-Magic-Commands.ipynb); Estos se pueden utilizar para cronometrar la ejecución repetida de fragmentos de código:

In [1]:
%timeit sum(range(100))

1.53 µs ± 47.8 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


Tenga en cuenta que debido a que esta operación es tan rápida, `%timeit` realiza automáticamente una gran cantidad de repeticiones.
Para comandos más lentos, `%timeit` se ajustará automáticamente y realizará menos repeticiones:

In [2]:
%%timeit
total = 0
for i in range(1000):
    for j in range(1000):
        total += i * (-1) ** j

536 ms ± 15.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


A veces repetir una operación no es la mejor opción.
Por ejemplo, si tenemos una lista que nos gustaría ordenar, es posible que una operación repetida nos engañe; ordenar una lista preordenada es mucho más rápido que ordenar una lista sin ordenar, por lo que la repetición distorsionará el resultado:

In [3]:
import random
L = [random.random() for i in range(100000)]
%timeit L.sort()

1.71 ms ± 334 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Para ello, la función mágica `%time` puede ser una mejor opción. También es una buena opción para comandos de ejecución más prolongada, cuando es poco probable que retrasos breves relacionados con el sistema afecten el resultado.
Cronometremos la clasificación de una lista sin clasificar y una lista previamente ordenada:

In [4]:
import random
L = [random.random() for i in range(100000)]
print("sorting an unsorted list:")
%time L.sort()

sorting an unsorted list:
CPU times: user 31.3 ms, sys: 686 µs, total: 32 ms
Wall time: 33.3 ms


In [5]:
print("sorting an already sorted list:")
%time L.sort()

sorting an already sorted list:
CPU times: user 5.19 ms, sys: 268 µs, total: 5.46 ms
Wall time: 14.1 ms


Observe cuánto más rápido se ordena la lista preclasificada, pero observe también cuánto más tiempo toma el tiempo con `%time` versus `%timeit`, ¡incluso para la lista preclasificada!
Esto es el resultado del hecho de que `%timeit` hace algunas cosas inteligentes bajo el capó para evitar que las llamadas al sistema interfieran con la sincronización.
Por ejemplo, evita la limpieza de objetos Python no utilizados (conocido como *recolección de basura*) que de otro modo podría afectar la sincronización.
Por esta razón, los resultados de "%timeit" suelen ser notablemente más rápidos que los resultados de "%time".

Para `%time`, al igual que con `%timeit`, el uso de la sintaxis mágica de celda `%%` permite cronometrar scripts multilínea:

In [6]:
%%time
total = 0
for i in range(1000):
    for j in range(1000):
        total += i * (-1) ** j

CPU times: user 655 ms, sys: 5.68 ms, total: 661 ms
Wall time: 710 ms


Para obtener más información sobre `%time` y `%timeit`, así como sus opciones disponibles, use la función de ayuda de IPython (por ejemplo, escriba `%time?` en el indicador de IPython).

## Scripts completos de creación de perfiles: %prun

Un programa se compone de muchas declaraciones individuales y, a veces, sincronizar estas declaraciones en contexto es más importante que sincronizarlas por sí solas.
Python contiene un generador de perfiles de código incorporado (sobre el cual puede leer en la documentación de Python), pero IPython ofrece una forma mucho más conveniente de utilizar este generador de perfiles, en la forma de la función mágica `%prun`.

A modo de ejemplo, definiremos una función simple que realiza algunos cálculos:

In [7]:
def sum_of_lists(N):
    total = 0
    for i in range(5):
        L = [j ^ (j >> i) for j in range(N)]
        total += sum(L)
    return total

Ahora podemos llamar a `%prun` con una llamada de función para ver los resultados perfilados:

In [8]:
%prun sum_of_lists(1000000)

         14 function calls in 0.932 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        5    0.808    0.162    0.808    0.162 <ipython-input-7-f105717832a2>:4(<listcomp>)
        5    0.066    0.013    0.066    0.013 {built-in method builtins.sum}
        1    0.044    0.044    0.918    0.918 <ipython-input-7-f105717832a2>:1(sum_of_lists)
        1    0.014    0.014    0.932    0.932 <string>:1(<module>)
        1    0.000    0.000    0.932    0.932 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}

El resultado es una tabla que indica, en orden de tiempo total en cada llamada a función, dónde pasa más tiempo la ejecución. En este caso, la mayor parte del tiempo de ejecución está en la lista de comprensión dentro de `sum_of_lists`.
A partir de aquí, podríamos empezar a pensar en qué cambios podríamos realizar para mejorar el rendimiento del algoritmo.

Para obtener más información sobre `%prun`, así como sus opciones disponibles, utilice la función de ayuda de IPython (es decir, escriba `%prun?` en el indicador de IPython).

## Perfilado línea por línea con %lprun

El perfilado función por función de `%prun` es útil, pero a veces es más conveniente tener un informe de perfil línea por línea.
Esto no está integrado en Python o IPython, pero hay un paquete `line_profiler` disponible para instalación que puede hacer esto.
Comience usando la herramienta de empaquetado de Python, `pip`, para instalar el paquete `line_profiler`:

```
$ pip instalar line_profiler
```

A continuación, puede usar IPython para cargar la extensión IPython `line_profiler`, que se ofrece como parte de este paquete:

In [9]:
%load_ext line_profiler

Ahora el comando `%lprun` creará un perfil línea por línea de cualquier función. En este caso, debemos decirle explícitamente qué funciones nos interesa perfilar:

In [10]:
%lprun -f sum_of_lists sum_of_lists(5000)

Timer unit: 1e-06 s

Total time: 0.014803 s
File: <ipython-input-7-f105717832a2>
Function: sum_of_lists at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def sum_of_lists(N):
     2         1          6.0      6.0      0.0      total = 0
     3         6         13.0      2.2      0.1      for i in range(5):
     4         5      14242.0   2848.4     96.2          L = [j ^ (j >> i) for j in range(N)]
     5         5        541.0    108.2      3.7          total += sum(L)
     6         1          1.0      1.0      0.0      return total

La información en la parte superior nos da la clave para leer los resultados: el tiempo se informa en microsegundos y podemos ver dónde pasa más tiempo el programa.
En este punto, es posible que podamos utilizar esta información para modificar aspectos del script y hacer que funcione mejor para nuestro caso de uso deseado.

Para obtener más información sobre `%lprun`, así como sus opciones disponibles, utilice la función de ayuda de IPython (es decir, escriba `%lprun?` en el indicador de IPython).

## Uso de la memoria de creación de perfiles: %memit y %mprun

Otro aspecto de la creación de perfiles es la cantidad de memoria que utiliza una operación.
Esto se puede evaluar con otra extensión de IPython, `memory_profiler`.
Al igual que con `line_profiler`, comenzamos instalando `pip` la extensión:

```
$ pip instalar memoria_profiler
```

Luego podemos usar IPython para cargarlo:

In [11]:
%load_ext memory_profiler

La extensión del perfilador de memoria contiene dos funciones mágicas útiles: `%memit` (que ofrece un equivalente de medición de memoria de `%timeit`) y `%mprun` (que ofrece un equivalente de medición de memoria de `%lprun`).
La función mágica `%memit` se puede utilizar de forma bastante sencilla:

In [12]:
%memit sum_of_lists(1000000)

peak memory: 141.70 MiB, increment: 75.65 MiB


Vemos que esta función utiliza unos 140 MB de memoria.

Para una descripción línea por línea del uso de la memoria, podemos usar la función mágica `%mprun`.
Desafortunadamente, esto solo funciona para funciones definidas en módulos separados en lugar del cuaderno en sí, por lo que comenzaremos usando la magia de la celda `%%file` para crear un módulo simple llamado `mprun_demo.py`, que contiene nuestra `sum_of_lists`. función, con una adición que hará que los resultados de nuestro perfil de memoria sean más claros:

In [13]:
%%file mprun_demo.py
def sum_of_lists(N):
    total = 0
    for i in range(5):
        L = [j ^ (j >> i) for j in range(N)]
        total += sum(L)
        del L # remove reference to L
    return total

Overwriting mprun_demo.py


Ahora podemos importar la nueva versión de esta función y ejecutar el perfilador de línea de memoria:

In [14]:
from mprun_demo import sum_of_lists
%mprun -f sum_of_lists sum_of_lists(1000000)

Filename: /Users/jakevdp/github/jakevdp/PythonDataScienceHandbook/notebooks_v2/mprun_demo.py

Line #    Mem usage    Increment  Occurences   Line Contents
     1     66.7 MiB     66.7 MiB           1   def sum_of_lists(N):
     2     66.7 MiB      0.0 MiB           1       total = 0
     3     75.1 MiB      8.4 MiB           6       for i in range(5):
     4    105.9 MiB     30.8 MiB     5000015           L = [j ^ (j >> i) for j in range(N)]
     5    109.8 MiB      3.8 MiB           5           total += sum(L)
     6     75.1 MiB    -34.6 MiB           5           del L # remove reference to L
     7     66.9 MiB     -8.2 MiB           1       return total

Aquí, la columna "Incremento" nos dice cuánto afecta cada línea al presupuesto total de memoria: observe que cuando creamos y eliminamos la lista "L", estamos agregando aproximadamente 30 MB de uso de memoria.
Esto se suma al uso de memoria en segundo plano del propio intérprete de Python.

Para obtener más información sobre `%memit` y `%mprun`, así como sus opciones disponibles, use la función de ayuda de IPython (por ejemplo, escriba `%memit?` en el indicador de IPython).